# Threading (потоки)

In [6]:
import threading
import logging
import time

def thread_function(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")


if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main: before creating thread")

    x = threading.Thread(target=thread_function, args=(1,))

    logging.info("Main: before running thread")

    x.start()

    logging.info("Main: waiting for the thread to finish")

    logging.info("Main: all done")

15:28:25: Main: before creating thread
15:28:25: Main: before running thread
15:28:25: Thread 1: starting
15:28:25: Main: waiting for the thread to finish
15:28:25: Main: all done


15:28:27: Thread 1: finishing


In [7]:
import threading
import logging
import time

from numpy import True_

def thread_function(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")


if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main: before creating thread")

    x = threading.Thread(target=thread_function, args=(1,), daemon=True)

    logging.info("Main: before running thread")

    x.start()

    logging.info("Main: waiting for the thread to finish")

    logging.info("Main: all done")

15:34:41: Main: before creating thread
15:34:41: Main: before running thread
15:34:41: Thread 1: starting
15:34:41: Main: waiting for the thread to finish
15:34:41: Main: all done


15:34:43: Thread 1: finishing


In [8]:
import threading
import logging
import time

def thread_function(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")


if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main: before creating thread")

    x = threading.Thread(target=thread_function, args=(1,), daemon=True)

    logging.info("Main: before running thread")

    x.start()

    logging.info("Main: waiting for the thread to finish")

    x.join() # ждем завершения потока

    logging.info("Main: all done")

15:38:17: Main: before creating thread
15:38:17: Main: before running thread
15:38:17: Thread 1: starting
15:38:17: Main: waiting for the thread to finish
15:38:19: Thread 1: finishing
15:38:19: Main: all done


In [14]:
import threading
import logging
import time

def thread_function(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")


if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    threads = []
    for i in range(3):
        logging.info(f"Main: create and start thread {i}")

        x = threading.Thread(target=thread_function, args=(i,))
        threads.append(x)
        x.start()
    logging.info("--------")
    for i, th in enumerate(threads):
        logging.info(f"Main: before joining thread {i}")
        th.join()
        logging.info(f"Main: thread {i} done")

15:48:48: Main: create and start thread 0
15:48:48: Thread 0: starting
15:48:48: Main: create and start thread 1
15:48:48: Thread 1: starting
15:48:48: Main: create and start thread 2
15:48:48: Thread 2: starting
15:48:48: --------
15:48:48: Main: before joining thread 0
15:48:50: Thread 0: finishing
15:48:50: Main: thread 0 done
15:48:50: Main: before joining thread 1
15:48:50: Thread 1: finishing
15:48:50: Main: thread 1 done
15:48:50: Thread 2: finishing
15:48:50: Main: before joining thread 2
15:48:50: Main: thread 2 done


In [19]:
import threading
import logging
import time
import concurrent.futures

def thread_function(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")


if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    n = 10
    with concurrent.futures.ThreadPoolExecutor(max_workers=n) as executor:
        executor.map(thread_function, range(n))

15:53:20: Thread 0: starting
15:53:20: Thread 1: starting
15:53:20: Thread 2: starting
15:53:20: Thread 3: starting
15:53:20: Thread 4: starting
15:53:20: Thread 5: starting
15:53:20: Thread 6: starting
15:53:20: Thread 7: starting
15:53:20: Thread 8: starting
15:53:20: Thread 9: starting
15:53:22: Thread 0: finishing
15:53:22: Thread 1: finishing
15:53:22: Thread 2: finishing
15:53:22: Thread 3: finishing
15:53:22: Thread 4: finishing
15:53:22: Thread 5: finishing
15:53:22: Thread 6: finishing
15:53:22: Thread 7: finishing
15:53:22: Thread 8: finishing
15:53:22: Thread 9: finishing


## Race conditions (состояние гонки)

In [20]:
class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info(f"Thread {name}: starting update")
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        logging.info(f"Thread {name}: finishing update")

In [21]:
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    db = FakeDatabase()
    logging.info(f"Testing update. Starting value is {db.value}")

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for i in range(2):
            executor.submit(db.update, i)
            
    logging.info(f"Testing update. Ending value is {db.value}")


16:07:05: Testing update. Starting value is 0
16:07:05: Thread 0: starting update
16:07:05: Thread 1: starting update
16:07:05: Thread 0: finishing update
16:07:05: Thread 1: finishing update
16:07:05: Testing update. Ending value is 1


In [24]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def locked_update(self, name):
        logging.info(f"Thread {name}: starting update")
        logging.info(f"Thread {name} about to lock")
        with self._lock:
            logging.info(f"Thread {name} has lock")
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.info(f"Thread {name} about to release lock")
        
        logging.info(f"Thread {name} after release")
        logging.info(f"Thread {name}: finishing update")

In [31]:
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    db = FakeDatabase()
    logging.info(f"Testing update. Starting value is {db.value}")

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for i in range(2):
            executor.submit(db.locked_update, i)
            
    logging.info(f"Testing update. Ending value is {db.value}")


16:19:59: Testing update. Starting value is 0
16:19:59: Thread 0: starting update
16:19:59: Thread 0 about to lock
16:19:59: Thread 1: starting update
16:19:59: Thread 0 has lock
16:19:59: Thread 1 about to lock
16:19:59: Thread 0 about to release lock
16:19:59: Thread 0 after release
16:19:59: Thread 1 has lock
16:19:59: Thread 0: finishing update
16:19:59: Thread 1 about to release lock
16:19:59: Thread 1 after release
16:19:59: Thread 1: finishing update
16:19:59: Testing update. Ending value is 2
